# Tic-Tac-Toe Q-Learning Visualization

This notebook visualizes the training progress and performance of the Q-learning agent.

In [ ]:
import sys
sys.path.insert(0, '../src')

import matplotlib.pyplot as plt
from game import TicTacToe
from agent import QLearningAgent
from train import train_self_play, TrainingStats
from evaluate import evaluate_agent, analyze_q_table

## Train the Agent

First, let's train an agent and collect statistics.

In [ ]:
# Create and train agent
agent = QLearningAgent(
    learning_rate=0.1,
    discount_factor=0.95,
    epsilon=1.0,
    epsilon_min=0.01,
    epsilon_decay=0.9995
)

# Train with statistics collection
stats = train_self_play(agent, num_episodes=50000, stats_interval=1000, verbose=True)

## Learning Curves

Visualize how the agent's win rates change over training.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Win rates over time
ax1 = axes[0, 0]
ax1.plot(stats.episodes, stats.x_wins, label='X Wins', color='blue', alpha=0.7)
ax1.plot(stats.episodes, stats.o_wins, label='O Wins', color='red', alpha=0.7)
ax1.plot(stats.episodes, stats.draws, label='Draws', color='green', alpha=0.7)
ax1.set_xlabel('Episode')
ax1.set_ylabel('Win Rate')
ax1.set_title('Win Rates During Self-Play Training')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_ylim(0, 1)

# Epsilon decay
ax2 = axes[0, 1]
ax2.plot(stats.episodes, stats.epsilons, color='purple')
ax2.set_xlabel('Episode')
ax2.set_ylabel('Epsilon')
ax2.set_title('Exploration Rate (Epsilon) Decay')
ax2.grid(True, alpha=0.3)

# Q-table growth
ax3 = axes[1, 0]
ax3.plot(stats.episodes, stats.q_table_sizes, color='orange')
ax3.set_xlabel('Episode')
ax3.set_ylabel('Q-Table Entries')
ax3.set_title('Q-Table Size Over Training')
ax3.grid(True, alpha=0.3)

# Draw rate trend
ax4 = axes[1, 1]
ax4.plot(stats.episodes, stats.draws, color='green', linewidth=2)
ax4.axhline(y=1.0, color='gray', linestyle='--', alpha=0.5, label='Perfect play draws')
ax4.set_xlabel('Episode')
ax4.set_ylabel('Draw Rate')
ax4.set_title('Draw Rate (Higher = More Optimal Play)')
ax4.legend()
ax4.grid(True, alpha=0.3)
ax4.set_ylim(0, 1)

plt.tight_layout()
plt.savefig('../training_curves.png', dpi=150)
plt.show()

## Evaluation Against Opponents

Test the trained agent against different opponents.

In [ ]:
# Evaluate against random
random_results = evaluate_agent(agent, 'random', num_games=1000)

print("=" * 50)
print("Performance vs Random Opponent (1000 games)")
print("=" * 50)
for role in ['as_x', 'as_o']:
    r = random_results[role]
    total = r['games']
    print(f"\n{role.replace('_', ' ').title()}:")
    print(f"  Wins: {r['wins']/total:.1%}")
    print(f"  Losses: {r['losses']/total:.1%}")
    print(f"  Draws: {r['draws']/total:.1%}")

In [ ]:
# Evaluate against minimax (optimal opponent)
minimax_results = evaluate_agent(agent, 'minimax', num_games=100)

print("=" * 50)
print("Performance vs Optimal (Minimax) Opponent (100 games)")
print("=" * 50)
for role in ['as_x', 'as_o']:
    r = minimax_results[role]
    total = r['games']
    print(f"\n{role.replace('_', ' ').title()}:")
    print(f"  Wins: {r['wins']/total:.1%}")
    print(f"  Losses: {r['losses']/total:.1%}")
    print(f"  Draws: {r['draws']/total:.1%}")

print("\nNote: Against a perfect opponent, the best possible result is 100% draws.")

In [ ]:
# Bar chart comparison
fig, ax = plt.subplots(figsize=(10, 6))

labels = ['vs Random\n(as X)', 'vs Random\n(as O)', 'vs Minimax\n(as X)', 'vs Minimax\n(as O)']
wins = [
    random_results['as_x']['wins'] / random_results['as_x']['games'],
    random_results['as_o']['wins'] / random_results['as_o']['games'],
    minimax_results['as_x']['wins'] / minimax_results['as_x']['games'],
    minimax_results['as_o']['wins'] / minimax_results['as_o']['games']
]
draws = [
    random_results['as_x']['draws'] / random_results['as_x']['games'],
    random_results['as_o']['draws'] / random_results['as_o']['games'],
    minimax_results['as_x']['draws'] / minimax_results['as_x']['games'],
    minimax_results['as_o']['draws'] / minimax_results['as_o']['games']
]
losses = [
    random_results['as_x']['losses'] / random_results['as_x']['games'],
    random_results['as_o']['losses'] / random_results['as_o']['games'],
    minimax_results['as_x']['losses'] / minimax_results['as_x']['games'],
    minimax_results['as_o']['losses'] / minimax_results['as_o']['games']
]

x = range(len(labels))
width = 0.25

ax.bar([i - width for i in x], wins, width, label='Wins', color='green', alpha=0.7)
ax.bar(x, draws, width, label='Draws', color='blue', alpha=0.7)
ax.bar([i + width for i in x], losses, width, label='Losses', color='red', alpha=0.7)

ax.set_ylabel('Rate')
ax.set_title('Agent Performance Against Different Opponents')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()
ax.set_ylim(0, 1)
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('../evaluation_results.png', dpi=150)
plt.show()

## Q-Table Analysis

Examine the learned Q-values.

In [ ]:
analysis = analyze_q_table(agent)

print("Q-Table Statistics")
print("=" * 40)
for key, value in analysis.items():
    if isinstance(value, float):
        print(f"{key}: {value:.4f}")
    else:
        print(f"{key}: {value}")

In [ ]:
# Distribution of Q-values
q_values = list(agent.q_table.values())

fig, ax = plt.subplots(figsize=(10, 5))
ax.hist(q_values, bins=50, color='steelblue', edgecolor='black', alpha=0.7)
ax.axvline(x=0, color='red', linestyle='--', alpha=0.7, label='Zero')
ax.set_xlabel('Q-Value')
ax.set_ylabel('Frequency')
ax.set_title('Distribution of Learned Q-Values')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../q_value_distribution.png', dpi=150)
plt.show()

## Save the Trained Agent

In [ ]:
# Save the agent
agent.save('../trained_agent.json')
print("Agent saved to ../trained_agent.json")
print(f"\nYou can now play against it by running: python src/play.py")